### Libraries

In [1]:
import math

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import scipy as sp

from scipy.optimize import fsolve, root_scalar

# Root Finding

Newtons Method: $$ x_{n+1} = x_n - \frac {f(x_n)}{f'(x_n)}$$

with a for loop

In [2]:
r=2

f = lambda x: 1-x-np.exp(-r*x)
fp = lambda x: -1+r*np.exp(-r*x)

tol = 1e-08
root = 0.4
for i in range(100000):
    root = root-f(root)/fp(root)  
    
# print(f"the root is {root}")

# Difference Equations

In [3]:
def logistic(x, r=3.9):
    return r*x*(1-x)

# logistic map for a few values of r

rs = [2.5, 3, 3.5, 3.9, 4.0, 4.1]
# rs = np.linspace(2.5, 4.0, 6)

# plt.figure(figsize=(12, 8))

for r in rs:
    xs = [0.1]

    for i in range(60):
        xs.append(logistic(xs[-1], r))
        
    # plt.subplot(2, 3, rs.index(r)+1)
    # plt.plot(xs, '--x')
    # plt.title(f"Logistic map with r={r}")

In [4]:
# bifurcation diagram for logistic map

rs = np.linspace(2, 4, 1000)

# plt.figure(figsize=(12, 8))

for r in rs:
    xs = [0.1]
    for i in range(1000):
        xs.append(logistic(xs[-1], r=r))

    # plt.plot([r]*len(xs[-20:]), xs[-20:], '.', markersize=1)

In [5]:
# finding 10-periodic solutions of the logistic map. these are groups

from scipy.optimize import fsolve
from functools import reduce 

period = 10


def composite(f, n):
    if n == 0:
        return lambda x: x
    return reduce(lambda f, g: lambda x: f(g(x)), [f]*n)


x = fsolve(lambda x: composite(logistic, period)(x)-x, 0.2)

# for i in range(period+1):
#     print(composite(logistic, i)(x))

# Numerical Differentiation

In [6]:
def fwd_diff(f, x, h=1e-8):
    return (f(x+h) - f(x)) / h

def ct_diff(f, x, h=1e-8):
    return (f(x+h) - f(x-h)) / (2*h)

# Regression

In [ ]:
def line(c,x):
    if x is np.ndarray:
        return np.array([c[0]*x + c[1] for x in x])
    return c[0]*x + c[1]

def llsq(pts):
    '''
        Returns the coefficients of the linear least squares fit of the points
    '''
    if type(pts) != np.ndarray:
        # could further handle if shape is wrong. if not right shape, transpose it.
        print('pts is not an array')
        pts = np.array(pts).T
        pts = np.array([pts[:,0], np.ones(len(pts))]).T
        ys = pts[:,1]
    else:
        # could further handle if shape is wrong. if not right shape, transpose it.
        ys = pts[:,1]
        pts = np.array([pts[:,0], np.ones(len(pts))]).T
    
    c = np.linalg.solve(pts.T@pts, pts.T@ys)
    plt.title('Linear Least Squares Demo')
    plt.plot(pts[:,0], line(c, pts[:,0]), label=f'slope = {c[0]:0.0f}, intercept = {c[1]:0.1f}')
    plt.plot(pts[:,0], ys, '.', label='data')
    plt.grid()
    plt.legend()
    return c

# PCA

In [ ]:
def pax(X):
    '''
        Return eigenvalues and eigenvectors of the covariance matrix
    '''
    cov = X.T@X
    vals, e = np.linalg.eig(cov)

    plt.plot(X[:, 0], X[:, 1], '.')
    plt.plot([0, e[0, 0]], [0, e[1, 0]], 'r')
    plt.plot([0, e[0, 1]], [0, e[1, 1]], 'g')
    plt.axis('equal')
    return vals, e